# Example analysis of a PPG/ECG waveform using the vital_sqi package

The following notebook shows an example of PPG/ECG waveform processing using the vital_sqi package. The aim of the package is to automate signal quality classification for PPG/ECG waveforms. It is achieved by computing various signal quality indices for each signal segment and using them to form a decision. 

## The pipeline can be briefly summarized as follows:
1. Load dataset under analysis
2. Preprocess and segment the dataset
3. Compute SQI for each dataset segment
4. Make decision for each segment

## Global Imports

In [ ]:
import numpy as np
import warnings
import os
import pandas as pd 
import matplotlib.pyplot as plt
import vital_sqi
from vital_sqi.data.signal_io import ECG_reader,PPG_reader

### Start by importing the signal via the PPG_reader function

The function expects a .csv or similar data format with named columns. The column names are used to separate between data column, timestamp columns and any additional information columns.
This returns a SignalSQI class that is compatible with other vital_sqi package functions, the main class members of interest are:
* signals:         an ndarray of shape (m, n) where m is the number of rows and n is the number of channels of the signal
* sqis:     an ndarray of shape (m, n) where m is the number of signal segments, n is the number of SQIs.
* sampling_rate:   sampling rate in hertz (Hz)


In [ ]:
from vital_sqi.pipeline.pipeline_highlevel import get_ppg_sqis

file_in = os.path.abspath('test_data/ppg_smartcare.csv') #FIle input location
sqi_dict = os.path.abspath('test_data/sqi_dict.json') #input dictionary -> which sqi features 
segments, signal_obj = get_ppg_sqis(file_in,
                                    signal_idx=['PLETH'], 
                                    timestamp_idx=['TIMESTAMP_MS'], 
                                    sqi_dict_filename = sqi_dict)

In [ ]:
signal_obj.sqis[0].describe()

### Get decision

In [ ]:
from vital_sqi.pipeline.pipeline_highlevel import get_qualified_ppg
rule_dict_filename = os.path.abspath('test_data/rule_dict_test.json') #  location dictionary contain threshold
ruleset_order = {3: 'skewness_sqi',
                  2: 'kurtosis_sqi',
                  1: 'perfusion_sqi'}
file_in = os.path.abspath('test_data/ppg_smartcare.csv') #FIle input location
sqi_dict = os.path.abspath('test_data/sqi_dict.json') #input dictionary -> which sqi features 
signal_obj = get_qualified_ppg(file_in,
                                    signal_idx=['PLETH'], 
                                    timestamp_idx=['TIMESTAMP_MS'], 
                                    sqi_dict_filename = sqi_dict,
                                    rule_dict_filename = rule_dict_filename,
                                    ruleset_order=ruleset_order,
                                    output_dir=None)